In [1]:
from IPython.display import Image

一开始我们通过语料训练了一个预训练模型 $\pi_{\text{PT}}$，然后有得到一个 $\pi_{\text{SFT}}$ 模型，接下来通过RLHF我们要得到一个 $\pi_{\text{RLHF}}$ 的模型。这其实是什么？是不断调整分布的过程，或者说是语言模型不断调整信念的过程。

In [3]:
# https://www.zhihu.com/question/629107126
Image(url='./imgs/kl-obj.png', width=400)

- RLHF
  $$
  R(x,y)=r(x,y)-\beta\log\frac{\pi(y|x)}{\pi_{ref}(y|x)}
  $$
    - Learning to summarize from human feedback
    - Fine-Tuning Language Models from Human Preferences
    - Training language models to follow instructions with human feedback
- 为什么 reward 中加入 K-L 项可以工作，以 TRL PPO 代码为例：
    - 将 kl item 放入负奖励，即采样时计算
        - reward = reward - kl * coef
    - 再利用 reward 计算 gae。这里 k-l 项的计算方法，在 TRL PPO 默认条件下
        - kl = log_prob - ref_log_prob
    - 将当前输出 token 的对数概率直接在 actor model (微调模型) 与 ref model (预训练模型)上求差值
    - 直觉上的理解
        - 当 log_prob < ref_log_prob 时，这里 kl 为负，reward 增加。可以视为 advantage 增加，那么会通过策略梯度增加当前输出 token 的 log_prob，那么会迫使 log_prob 有靠近 ref_log_prob 的趋势（不要小太多）。
        - 当 log_prob > ref_log_prob 时，这里 kl 为正，reward 降低。可以视为 advantage 降低，那么会通过策略梯度减小当前输出 token 的 log_prob，那么同样会迫使 log_prob 有靠近 ref_log_prob 的趋势（不要大太多）。
        - 所以，kl item 加入奖励，确实会导致 log_prob 不会太偏离 ref_log_prob，也就是使当前更新模型 actor model 的输出每个 token 的概率不会偏离 reference model 预训练模型太远，就达到了控制微调的预期效果。

### token 级别的 reward 的传递

$$
\begin{split}
&Q_t=\sum_{k=t}^T\gamma^{k-t}r_k\\
&Q_t=r_t+\gamma Q_{t+1}
\end{split}
$$
- 未来的reward折现；
- 假如一个序列长度为 3，$\gamma=0.9$，奖励 $\mathbf r=[0,0,3]$（从后往前算）
    - $Q_2=3$
    - $Q_1=r_1+0.9r_2=2.7$
    - $Q_0=r_0+0.9r_1+0.9^2r_2=2.43$